In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm
from torch.autograd import Variable
from datetime import datetime, timedelta
import models
import data_preparation

In [7]:
X_train = np.load('./data/X_train_surge_new.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge_new.npz')

In [8]:
model = models.PressureEncorderLinear()
device = torch.device('cuda')
model = model.to(device)

epochs = 100
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

In [18]:
train_dataloader, val_dataloader = data_preparation.data_prepare_pretrain_semifull_small(X_train, Y_train)

In [19]:
for epoch in range(epochs):
    model.train()
    for x1, x2, y in tqdm(train_dataloader, total = len(train_dataloader), leave=False):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model((x1, x2))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    scheduler.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, y in tqdm(val_dataloader, total = len(val_dataloader), leave = False):
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            x1 = x1.type(torch.cuda.FloatTensor)
            x2 = x2.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.FloatTensor)
            pred = model((x1, x2))
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_dataloader))
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/630 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
dl = data_preparation.data_prepare_pretrain_semifull_small(X_test, Y_train, train=False)

In [16]:
model.eval()

ys = []
with torch.no_grad():
    for x1, x2 in tqdm(dl, total = len(dl), leave = False):
        x1, x2 = x1.to(device), x2.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        ys.append(model((x1, x2)).to(torch.device('cpu')))
surge_test = np.concatenate(ys, axis=0)

  0%|          | 0/64 [00:00<?, ?it/s]

In [17]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=surge_test, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('Y_test_benchmark_semi_full_1.csv', index_label='id_sequence', sep=',')